In [1]:
s = '-i data/sic_hierarchy -m xmlcnn -o temp -l binary'.split(' ')

# IMPORTS

In [2]:
# basic
import argparse
import os,datetime
# data input
import numpy as np
import scipy.sparse
# metric and loss function
import keras.backend as K
import tensorflow as tf
from keras.metrics import categorical_accuracy, binary_accuracy, top_k_categorical_accuracy
# embedding
from keras.initializers import Constant
from keras.layers import Embedding
# models
from keras.layers import Dense, Input, Flatten, Concatenate, Conv1D, MaxPooling1D, Dropout
from keras.layers import CuDNNLSTM, Bidirectional, TimeDistributed, Lambda, Softmax
from keras.initializers import Constant
from keras.models import Model
# misc
from tools.MyClock import MyClock
# save things
import pandas as pd
from keras.callbacks import CSVLogger

Using TensorFlow backend.


In [3]:
def Coloured(string):
    return "\033[1;30;43m {} \033[0m".format(string)

# ARGPARSE

In [4]:
parser = argparse.ArgumentParser(description = 'run baseline models')
parser.add_argument('-i','--input', required = True, type = str, help = 'input directory e.g. ./data/dl_amazon_1/')
parser.add_argument('-m','--model', required = True, type = str, help = 'model, one in: xmlcnn, attentionxml, attention,')
parser.add_argument('-l','--loss', required = True, type = str, help = "loss type, categorical or binary ")
parser.add_argument('-o','--output', required = True, type = str, help = 'output directory')
parser.add_argument('--mode', required = True, type = str, help = 'cat,hierarchy')
parser.add_argument('--epoch', default = 5, type = int, help = 'epochs')
parser.add_argument('--batch_size', default = 0, type = int, help = 'batch size')
parser.add_argument('--save_weights', default = True, action = 'store_true', help = 'save trained weights')
parser.add_argument('--save_model', default = True, action = 'store_true', help = 'save trained model architecture')
parser.add_argument('--save_prediction', default = True, action = 'store_true', help = 'save top 10 prediction and corresponding probabilities (not implemented yet)')
args = parser.parse_args(s)

# argparse validation
default_batch_size = {'xmlcnn':128,'attentionxml':20,'attention':25}
if not os.path.exists(args.input):
    raise Exception('Input path does not exist: {}'.format(args.input))
if args.model not in default_batch_size.keys():
    raise Exception('Unknown model: {}'.format(args.model))
if args.loss not in ['binary','categorical']:
    raise Exception('Unknown loss: {}'.format(args.loss))
if args.mode not in ['caat','hierarchy']:
    raise Exception('Unknown mode: {}'.formate(args.mode))

IN_DIR = args.input
if not args.batch_size:
    args.batch_size = default_batch_size[args.model]
if not os.path.exists(args.output):
    os.mkdir(args.output)
    print(Coloured("Create Output Directory: {}".format(args.output)))
OUT_DIR = os.path.join(
    args.output,
    datetime.datetime.now().strftime('%y%m%d_%H%M%S_{}'.format(args.model)),
)
if not os.path.exists(OUT_DIR):
    os.mkdir(OUT_DIR)

# INPUT

In [5]:
def get_input(in_dir,mode):
    x_train = np.load(os.path.join(in_dir,'x_train.npy'))
    dirs = [os.path.join(in_dir,d) for d in sorted(os.listdir(in_dir)) if d.startswith('y_train_{}'.format(mode))]
    y_trains = [scipy.sparse.load_npz(d).todense() for d in dirs]

    x_test = np.load(os.path.join(in_dir,'x_test.npy'))
    dirs = [os.path.join(in_dir,d) for d in sorted(os.listdir(in_dir)) if d.startswith('y_test_{}'.format(mode))]
    y_tests = [scipy.sparse.load_npz(d).todense() for d in dirs]
    return x_train,y_trains,x_test,y_tests

# EMBEDDING LAYER

In [6]:
def get_embedding_layer():
    embedding_matrix = np.load(os.path.join(IN_DIR,'embedding_matrix.npy'))
    num_words, embedding_dim = embedding_matrix.shape
    embedding_layer = Embedding(num_words,
                                embedding_dim,
                                embeddings_initializer=Constant(embedding_matrix),
                                trainable=False)
    return embedding_layer

# MODELS

In [7]:
def apply_attention(inputs):
    input1, input2 = inputs
    outer_product = tf.einsum('ghj, ghk -> gjk', input1, input2)
    return outer_product

def get_model(model_name, max_sequence_length, labels_dims):
    embedding_layer = get_embedding_layer()
    sequence_input = Input(shape=(max_sequence_length,), dtype='int32')
    embedded_sequences = embedding_layer(sequence_input)
    if model_name == 'xmlcnn':
        filter_sizes = [2,4,8]
        pooling_units = 32
        bottle_neck = 512
        convs = []
        for fsz in filter_sizes:
            l = Conv1D(filters = 128, kernel_size = fsz, strides = 2, activation = 'relu')(embedded_sequences)
            s = int(l.shape[-2])
            pool_size = s//pooling_units
            l = MaxPooling1D(pool_size,padding = 'same')(l)
            l = Flatten()(l)
            convs.append(l)
        x = Concatenate(axis=-1)(convs)
        x = Dense(bottle_neck, activation = 'relu')(x)
        x = Dropout(0.5)(x)
        outs = []
        for i,labels_dim in enumerate(labels_dims):
            outs.append(Dense(labels_dim, activation = None, name = 'H{}'.format(i))(x))
    elif model_name in ['attentionxml','attention']:
        labels_dim = sum(labels_dims)
        if model_name == 'attentionxml':
            # with lstm
            x = Bidirectional(CuDNNLSTM(512, return_sequences=True))(embedded_sequences)
        else:
            # without lstm
            x = embedded_sequences
        attention = Dense(labels_dim,activation=None,name='attention_dense')(x)
        attention = Softmax(axis=1,name='attention_softmax')(attention)
        x = Lambda(apply_attention,name = 'apply_attention')([x, attention])
        x = Lambda(lambda x:K.permute_dimensions(x,(0,2,1)),name='transpose')(x)
        x = TimeDistributed(Dense(512,activation='relu'))(x)
        x = Dropout(0.5)(x)
        x = TimeDistributed(Dense(256,activation='relu'))(x)
        x = Dropout(0.5)(x)
        x = TimeDistributed(Dense(1,activation=None))(x)
        x = Lambda(lambda x:K.squeeze(x,axis=-1))(x)
        outs = []
        start = 0
        for i,labels_dim in enumerate(labels_dims):
            outs.append(Lambda(lambda x:x[:,start:start+labels_dim],name = 'H{}'.format(i))(x))
            start+=labels_dim
    else:
        raise Exception('Invalid model_name : {}'.format(model_name))
    return Model(sequence_input, outs)

# LOSSES

In [8]:
def binary_cross_entropy_with_logits(y_true, y_pred):
    return K.mean(K.binary_crossentropy(y_true,y_pred,from_logits=True),axis=-1)
def categorical_cross_entropy_with_logits(y_true, y_pred):
    return K.mean(K.categorical_crossentropy(y_true,y_pred,from_logits=True),axis=-1)
loss_dict = {'binary':binary_cross_entropy_with_logits,
             'categorical':categorical_cross_entropy_with_logits,}

# METRICS

In [9]:
def binary_accuracy_with_logits(y_true, y_pred):
    return K.mean(K.equal(y_true, K.tf.cast(K.less(0.0,y_pred), y_true.dtype)))
def pAt1(y_true,y_pred):
    return categorical_accuracy(y_true, y_pred)
def pAt5(y_true,y_pred):
    return top_k_categorical_accuracy(y_true, y_pred, k=5)

# MISC

In [10]:
def save_predictions(model,x_test,y_tests,out_dir):
    print('SAVE PREDICTIONS TO : {}'.format(out_dir))
    out_probs = model.predict(x_test,verbose=1)
    ind_dirs = [os.path.join(out_dir,'pred_outputs{}.txt'.format(i)) for i in range(len(y_tests))]
    log_dirs = [os.path.join(out_dir,'pred_logits{}.txt'.format(i)) for i in range(len(y_tests))]
    f_ind = [open(ind_dir,'ab') for ind_dir in ind_dirs]
    f_log = [open(log_dir,'ab') for log_dir in log_dirs]
    for i,out_prob in enumerate(out_probs):
        ind = np.argsort(out_prob,axis=1)[:,:-11:-1]
        logits = np.take_along_axis(out_prob, ind, axis=1)
        np.savetxt(f_ind[i],ind,fmt='%d')
        np.savetxt(f_log[i],logits,fmt='%1.3f')
    for f in f_ind + f_log:
        f.close()

# MAIN

In [13]:
csv_logger = CSVLogger(os.path.join(OUT_DIR,'train.log'),append=False)
# inputs
x_train,y_trains,x_test,y_tests = get_input(IN_DIR,args.mode)
_,max_sequence_length = x_train.shape
labels_dims = [l.shape[-1] for l in y_tests]
model = get_model(args.model, max_sequence_length, labels_dims)
model.compile(loss = loss_dict[args.loss],
              optimizer = 'adam',
              metrics = [pAt1,pAt5])
model.fit(x_train, y_trains,
          batch_size = args.batch_size,
          epochs = args.epoch,
          validation_data = (x_test, y_tests),
          callbacks = [csv_logger],
          shuffle = True,
         )

W0725 15:08:10.132068 140424970528576 deprecation_wrapper.py:119] From /home/angela/env/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0725 15:08:10.142550 140424970528576 deprecation_wrapper.py:119] From /home/angela/env/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0725 15:08:10.441274 140424970528576 deprecation_wrapper.py:119] From /home/angela/env/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0725 15:08:10.456938 140424970528576 deprecation_wrapper.py:119] From /home/angela/env/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0725 15:08:10.513536 140424970

Train on 572101 samples, validate on 142756 samples
Epoch 1/5
572101/572101 [==============================] - 94s 165us/step - loss: 0.1576 - H0_loss: 0.1025 - H1_loss: 0.0351 - H2_loss: 0.0106 - H3_loss: 0.0094 - H0_pAt1: 0.6588 - H0_pAt5: 0.9062 - H1_pAt1: 0.5498 - H1_pAt5: 0.7786 - H2_pAt1: 0.3431 - H2_pAt5: 0.5779 - H3_pAt1: 0.3219 - H3_pAt5: 0.5583 - val_loss: 0.1338 - val_H0_loss: 0.0905 - val_H1_loss: 0.0292 - val_H2_loss: 0.0075 - val_H3_loss: 0.0066 - val_H0_pAt1: 0.6962 - val_H0_pAt5: 0.9310 - val_H1_pAt1: 0.6123 - val_H1_pAt5: 0.8408 - val_H2_pAt1: 0.4577 - val_H2_pAt5: 0.7001 - val_H3_pAt1: 0.4341 - val_H3_pAt5: 0.6863
Epoch 2/5
572101/572101 [==============================] - 90s 158us/step - loss: 0.1351 - H0_loss: 0.0906 - H1_loss: 0.0298 - H2_loss: 0.0078 - H3_loss: 0.0068 - H0_pAt1: 0.7024 - H0_pAt5: 0.9299 - H1_pAt1: 0.6113 - H1_pAt5: 0.8356 - H2_pAt1: 0.4399 - H2_pAt5: 0.6880 - H3_pAt1: 0.4168 - H3_pAt5: 0.6736 - val_loss: 0.1316 - val_H0_loss: 0.0896 - val_H1_loss:

# save things

In [14]:
if args.save_weights:
    model.save_weights(os.path.join(OUT_DIR,'weights.h5'))
if args.save_model:
    with open(os.path.join(OUT_DIR,'model.json'),'w') as f:
        f.write(model.to_json())
if args.save_prediction:
    save_predictions(model,x_test,y_tests,OUT_DIR)
pd.DataFrame.from_dict([vars(args)]).to_csv(os.path.join(OUT_DIR,'args.csv'))

SAVE PREDICTIONS TO : temp/190725_150715_xmlcnn
142756/142756 [==============================] - 13s 88us/step
